## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [11]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import matplotlib.pyplot as plt

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability.

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Compute the error of the estimated pulse rate and the coresponding cofidence level.

    Args:
        data_f1: Name of the file that contain signal data
        ref_f1: Name of the file that contain reference data of the ground-truth of heart rate

    Returns:
        errors: a numpy arrray contains the difference between ground-trueth and the estimated pulse rate 
        confidence: a numpy array contains the confidence level of the estimated pulse rate
        
    """

    fs = 125

    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    ref_bpm = sp.io.loadmat(ref_fl)['BPM0']

    # Bandpass filter all signals between 40-240 BPM
    ppg = bandpass_filter(ppg, fs)
    accx = bandpass_filter(accx, fs)
    accy = bandpass_filter(accy, fs)
    accz = bandpass_filter(accz, fs)


    # define windows
    window_length = 8 * fs    # 8 sec >> no of sample
    window_shift = 2 * fs     # 6 sec >> no of sample

    n = window_length*10
    freqs = np.fft.rfftfreq(n, 1/fs)

    j = 0  # index for extracting ground true
    errors, confidence = [], []

    tolerance = 2   #min difference betweeen estimated pulse rate and vibration rate of accelerator

    # take windows of signals
    for i in range(0, len(ppg) - window_length, window_shift):
        ppg_window = ppg[i: i + window_length]
        accx_window = accx[i: i + window_length]
        accy_window = accy[i: i + window_length]
        accz_window = accz[i: i + window_length]

        # L2-Norms of acc
        acc_window = np.sqrt(accx_window**2 + accy_window**2 + accz_window**2)

        # FFT of ppg signal and the top 5 possible pulse rate
        ppg_bpm, ppg_fft = signal_process(ppg_window, freqs, n, 2)
        
        # Vibration rates of accelerators
        accx_bpm = signal_process(accx_window, freqs, n, 1)
        accy_bpm = signal_process(accy_window, freqs, n, 1)
        accz_bpm = signal_process(accz_window, freqs, n,1)
        acc_bpm = signal_process(acc_window, freqs, n, 1)
        
        # if frequency of ppg = acc, use the next possible pulse rate
        for k in range(4):
            est_bpm = ppg_bpm[k] 
            if (abs(est_bpm - acc_bpm) < tolerance or abs(est_bpm - accx_bpm) < tolerance or abs(est_bpm - accy_bpm) < tolerance or abs(est_bpm - accz_bpm) < tolerance):
                est_bpm = ppg_bpm[k+1]
            else:
                break

        # ground truth of pu;se rate
        true_bpm = ref_bpm[j].item()
        j += 1     
        
        # Compute error
        error = np.abs(true_bpm-est_bpm)
        errors.append(error)

        # Compute an estimation confidence, spectrum near the peak / total spectrum
        peak_range = (freqs > (est_bpm - 5)/60) & (freqs < (est_bpm+5)/60)
        peak_energy = np.sum(ppg_fft[peak_range])
        total_energy = np.sum(ppg_fft)
        peak_to_total = peak_energy / total_energy
        confidence.append(peak_to_total)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.array(errors), np.array(confidence)
    return errors, confidence


def bandpass_filter(signal, fs):
    """
    Bandpass filter the signal between low bpm and high bpm

    Args:
        signal: a numpy array of unfiltered signals.
        fs: int, signal sample rate

    Returns:
        a numpy array of filtered signals.
    """

    pass_band=(40/60.0, 240/60.0)
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)

    return sp.signal.filtfilt(b, a, signal)


def signal_process(signal, freqs, n, signal_type = 1):
    """
    FFT of signal, find peaks in frequency domain

    Args:
        signal: numpy arrays of signal
        freqs: numpy array of frequency in frequency domain
        n: int, number of points in frequency domain
        signal_type: 1, accelerator, 2, ppg

    Returns:
        accelerator: vibration rate
        ppg: 5 estimated pulse rates and FFT of ppg
    """

    # Fourier Transform of signal
    signal_fft = np.abs(np.fft.rfft(signal,n))

    # find peaks for signal  in freqeucny domain
    signal_peaks, _ = sp.signal.find_peaks(signal_fft)

    # arrange peaks in descending order
    signal_magnitude = []

    for peak in signal_peaks:
        signal_magnitude.append(signal_fft[peak])

    signal_magnitude = sorted(signal_magnitude, reverse = True)

    # return top 5 estimated pulse rate and FFT of ppg
    if signal_type == 2:
        signal_bpm = []
        for i in range(5):
            signal_bpm.append(freqs[list(signal_fft).index(signal_magnitude[i])] *60)
        return  signal_bpm, signal_fft

    # retun vibration rate of accelerator
    signal_bpm = freqs[list(signal_fft).index(signal_magnitude[0])] *60
    return signal_bpm


In [12]:
Evaluate()

7.3614677320180917

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code Description**
In this project, the algorithm can estimate the pulse rate from the signals of ppg sensor and accelerators. The signals data should be saved in .mat files. Providing the paths of files, pulse rate in every 2 seconds will be calculated.  

**Data Description**
The signals are taken from the Troika dataset. The original dataset has 2-channels PPG signals, 1-channel ECG signal and 3-axis accelerator signals. But, only 1 PPG signals and 3 accelerator signals are used in this project. In order to have a more complicated algorithm, the full dataset can be used. In addition, other data like position from GPS can also be included.

**Algorithhm Description**
The algorithm transforms the PPG signal from time domain to frequency domain. Frequency with the highest amplitude is the estimated pulse rate. The PPG signals were affected by the arm motion. So, the vibration rate were also esitmated from the accelerators signals. If the estimated pulse rate and the vibration rate are equal, the next strongest frequency in PPG signal is the estimated pulse.
The pulse rate were estimated in every 2 seconds. The corresponding condifence level and error were also calculated. If the pulse rate was the same as the frequency of the arm motion, the algorithm may reject the estimated reuslt and select a wrong pulse rate.

**Algorithm Performance**
The confidence value was estimated by the sum of amplitude near the peak in frequency domain over the whole spectrum. The highest the confidence value the better the estimated pulse rate is. The error between the ground-truth and the estimated pulse rate was calculated. In this dataset, the mean error of the best 90% of estimates is 7.4 BPM. The result is not bad. It is believed that the algorithm can be applied to other datasets.


-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 